In [1]:
import argparse
import codecs
import json
import re
import platform
import csv
from itertools import combinations

In [2]:
language = ['Korean', 'English', 'Japanese', 'Mandarin', 'Traditional Chinese',
            'Vietnamese', 'German', 'French', 'Other language', 'Spanish',
            'Indonesian', 'Russian', 'Arabic', 'Thai', 'Swedish', 'Dutch',
            'Hebrew', 'Tagalog', 'Portuguese(Brazil)', 'Cantonese', 'Italian',
            'Esperanto', 'Hawaiian', 'Afrikaans', 'Mongolian', 'Hindi', 'Polish',
            'Finnish', 'Greek', 'Bihari', 'Farsi', 'Urdu', 'Turkish', 'Portuguese(Portugal)',
            'Bulgarian', 'Norwegian', 'Romanian', 'Albanian', 'Ukrainian', 'Catalan',
            'Latvian', 'Danish', 'Serbian', 'Slovak', 'Georgian', 'Hungarian', 'Malaysian',
            'Icelandic', 'Latin', 'Laotian', 'Croatian', 'Lithuanian', 'Bengali', 'Tongan',
            'Slovenian', 'Swahili', 'Irish', 'Czech', 'Estonian', 'Khmer', 'Javanese', 'Sinhalese',
            'Sanskrit', 'Armenian', 'Tamil', 'Basque', 'Welsh', 'Bosnian', 'Macedonian', 'Telugu',
            'Uzbek', 'Gaelic', 'Azerbaijanian', 'Tibetan', 'Panjabi', 'Marathi', 'Yiddish', 'Ainu',
            'Haitian', 'Slavic']
color_tags = ["f-red", "f-blue", "f-bold"]
sline_tag = "sline]"

In [12]:
def main():
    assert platform.python_version_tuple()[0] == '3', 'This program supports only python3'
    args = parse_args()
    data_num = 0
    error_num = 0
    with codecs.open(args.data_path, 'r', encoding='utf8') as f:
        for line in f:
            data_num += 1
            try:
                jsonData = json.loads(line, strict=False)
                l2_langs, l1_lang = jsonData[2], jsonData[3]
                orig_sents, corr_sents = jsonData[4], jsonData[5]
                 
            except:
                error_num += 1
                pass

In [45]:
f = codecs.open(r'C:\Users\datat\Documents\GitHub\anlp-at2-gpt45\lang-8-20111007-L1-v2.dat', 'r', encoding='utf8')

In [46]:
data_num = 0
error_num = 0
orig_sentences = []
corr_sentences = []

for line in f:
    data_num +=1
    try:
        jsonData = json.loads(line, strict=False)
        l2_langs, l1_lang = jsonData[2], jsonData[3]
        orig_sents, corr_sents = jsonData[4], jsonData[5]
        orig_sentences.append(jsonData[4])
        corr_sentences.append(jsonData[5])
    except:
        error_num += 1
        pass



In [48]:
print(orig_sentences[1])

['방학때 했던 일들 What I did during my summer vacation', 'Lang-8에는 Soyo씨가 "민소주씨,you have to write more Korean entry.:)"말했어요 하하하 아주 맞는 말이에요.', '그래서 이 문장 써되었어요.', 'Lang-8\'s Soyo said this "민소주씨,you have to write more Korean entry.:)" Haha, and she\'s right.', "That's how I ended up writing this entry.", '요즘에 한국어 공부를 많이 한 했어요.', '이번 방학은 느긋한 방학이 되었어요.', '그래도 매일 한 문장 쓰야하죠?', '그럼 맨날 쓰면 좋은 습관이 될거시죠.', 'ㅎㅎㅎㅎ 공부 많이 하지않았때문데, Lang-8 친구에서 배웠던거 단어와 표현이 다 잊혀버렸어요 ㅠ ㅠ 그래서 지금 더 열심히 공부해야겠다!', "Recently I haven't been studying Korean.", "This summer vacation I've been really lazy.", 'Still, I should at least write a passage daily right?', "Then it'll become a good habit.", "I have forgotten almost all the words and expressions I've learned from my Lang-8 friends T T that's why I have to work even harder now!", '그럼 뱅학때 내가 뭐 하고있었어요?', '이모랑 요리 좀 배웠거든요.', '드디어!', '재가 아직 복잡하는 음식이 만들 수 가없지만, 야채이나 간단하는거 할 수 있어요 ^^ 너무 즐거워요 요리 할 때.', '이모랑 얘기도 하고, 노래도 하고...', '가끔은 현재도 부엌에서 도와주고, 너무 행복해요 이런 날은.', 'Then what have I been 

In [4]:
def make_sent_pair(orig_sents, corr_sents, args):
    outputs = []
    for i, orig_sent in enumerate(orig_sents):
        orig_sent = orig_sent.replace('\t', ' ')
        if len(corr_sents[i]) > 0:
            tag_err = False
            for corr_sent in corr_sents[i]:
                corr_sent = corr_sent.replace('\t', ' ')
                text, tag_err = delete_tags_color(corr_sent, tag_err, args)
                if sline_tag in text:
                    text, tag_err = delete_tags_sline(text, tag_err, args)
                if not tag_err and text != "":
                    output = orig_sent + "\t" + text
                    outputs.append(output)
        else:
            output = orig_sent + "\t" + orig_sent
            outputs.append(output)

    return outputs

In [5]:
def delete_tags_sline(text, tag_err, args):
    s_sline = "[sline]"
    e_sline = "[/sline]"
    if args.tags:
        return text
    words = text.split(" ")
    total_s = total_e = 0
    output_lists, tmp_list = [], []
    for word in words:
        num_s = word.count(s_sline)
        num_e = word.count(e_sline)

        total_s += num_s
        total_e += num_e
        tmp_list.append(word)
        if total_s == 0 and total_e == 0:
            output_lists.append(word)
            tmp_list = []
        elif total_s == total_e:
            tmp_text = " ".join(tmp_list)
            tmp_text = re.sub(r"\[sline\](.*)\[\/sline\]", r"", tmp_text)
            if tmp_text != "":
                output_lists.append(tmp_text)
            total_s = total_e = 0
            tmp_list = []
    text = " ".join(output_lists)

    if sline_tag in text:
        tag_err = True

    text = re.sub(r'\s+', ' ', text)
    return text, tag_err

In [7]:
def delete_tags_color(text, tag_err, args):
    if args.tags:
        return text
    text = replace_tags(text)

    if text == None:
        text = ""
    for tag in color_tags:
        s = "\[" + tag + "\]"
        e = "\[\/" + tag + "\]"
        text = re.sub(r"%s" % s, r"", text)
        text = re.sub(r"%s" % e, r"", text)
        if tag in text:
            tag_err = True

    return text, tag_err

In [8]:
def replace_tags(s):
    s = s.replace("[赤]", "[f-red]")
    s = s.replace("[/赤]", "[/f-red]")
    s = s.replace("[青]", "[f-blue]")
    s = s.replace("[/青]", "[/f-blue]")
    return s

In [11]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("-d", "--data", dest="data_path", type=str, metavar='<str>', required=True, help="The path to the data set")
    parser.add_argument("-l2", "--learn-lang", dest="l2", type=str, metavar='<str>', required=False, default='English', help="L2 language")
    parser.add_argument("-l1", "--native-lang", dest="l1", type=str, metavar='<str>', required=False, default=None, help="L1 language")
    parser.add_argument("-tags", "--remain-tags", dest="tags", default=False, action='store_true', help="If you want to remain tags (e.g. [f-red]), please use this option")

    args = parser.parse_args()

    assert args.l2 in language
    if args.l1 != None:
        assert args.l1 in language

    return args

In [ ]:
data = 